# Large Language Models

_I didn’t have time to write a short letter, so I wrote a long one instead_ - Mark Twain

We introduce large language models (LLMs) through a financial natural language processing (NLP) task: summarizing the *Quantitative and Qualitative Disclosures About Market Risk* sections of 10-K reports. To assess performance, we compare the overlap and readability of summaries generated by GPT-4o-mini, a proprietary closed-source model, and DeepSeek-R1-14B, an open-source model that can be downloaded and run locally. Small language models, particularly those trained using techniques like **distillation**, can closely approximate the performance of larger models while offering lower latency and reduced memory requirements.

In [1]:
# By: Terence Lim, 2020-2025 (terence-lim.github.io)
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import textwrap
from pprint import pprint
from rouge_score import rouge_scorer
from tqdm import tqdm
from finds.database import SQL, RedisDB
from finds.unstructured import Edgar
from finds.structured import BusDay, CRSP, PSTAT
from finds.readers import Sectoring
from secret import paths, credentials
VERBOSE = 0

In [2]:
sql = SQL(**credentials['sql'], verbose=VERBOSE)
user = SQL(**credentials['user'], verbose=VERBOSE)
bd = BusDay(sql)
rdb = RedisDB(**credentials['redis'])
crsp = CRSP(sql, bd, rdb, verbose=VERBOSE)
pstat = PSTAT(sql, bd, verbose=VERBOSE)
ed = Edgar(paths['10X'], zipped=True, verbose=VERBOSE)

## OpenAI GPT models

Large language models are built using transformer-based deep learning architectures and pre-trained on massive text corpora. GPT models, short for Generative Pre-trained Transformers, use an autoregressive approach to learn the structure of language by predicting the next token given the previous ones. The transformer architectures allows these models to capture long-range dependencies in text, making them particularly powerful for understanding context and generating fluent text. Modern LLMs extend this base with techniques like instruction tuning and reinforcement learning from human feedback (RLHF), which improve their usability and alignment with human intent.  

- **Pre-training** teaches the model general language patterns from large amounts of raw text data. This process builds a foundational base that can be fine-tuned for specific tasks later.

- **Instruction tuning** guides the model to follow specific types of tasks or instructions.  

- **RLHF** improves output quality by training the model to reflect human preferences.

OpenAI’s GPT series, from GPT-2 to GPT-o3, has demonstrated increasingly powerful capabilities due to the scale of their parameters and training data, containing millions or billions, and now trillions of adjustable weights in their deep neural networks. GPT-3 represented a fundamental shift in AI, demonstrating how scaling models alone could achieve generalization. It also introduced *In-Context Learning*, allowing models to learn from examples in the prompt without fine-tuning. GPT-4 expanded the context length to 128K **tokens** (which are how LLMs represent the fundamental units of text, which can be as small as single characters or as large as whole words), significantly improving its ability to understand and summarize long documents. These models, however, are only available through proprietary APIs.

In [3]:
gpt_name = "gpt-4o-mini"

### LangChain framework

A modular framework for building applications with language models, such as **LanChain** simplifies the process of integrating language models with external data sources and other AI tools. It abstracts over the underlying LLM API (OpenAI, Ollama, etc.) and allows users to create chains of prompts, tools, and logic for custom NLP workflows. 




In [4]:
# Initializes an OpenAI model using LangChain. temperature=0 ensures deterministic outputs
from langchain_openai import ChatOpenAI
gpt_model = ChatOpenAI(model_name=gpt_name, temperature=0, **credentials['openai'])
pprint(gpt_model.to_json())

{'id': ['langchain', 'chat_models', 'openai', 'ChatOpenAI'],
 'kwargs': {'model_name': 'gpt-4o-mini',
            'openai_api_key': {'id': ['OPENAI_API_KEY'],
                               'lc': 1,
                               'type': 'secret'},
            'temperature': 0.0},
 'lc': 1,
 'name': 'ChatOpenAI',
 'type': 'constructor'}


**Temperature** controls randomness in generation: lower values yield more deterministic responses, while higher values lead to more creative or diverse outputs.


### Open-source models

A large language model consists of three key components:

- Architecture: The structure of the model (e.g., Transformer-based).
- Weights: The learned parameters that define the model's behavior.
- Training code & data: The scripts and datasets used to train the model.

LLMs are categorized as:

- **Closed models**: API-only, no access to weights or training data (e.g., GPT-4).
- **Open models**: Model weights are available, but full training details are not (e.g., LLaMA, Qwen).
- **Open-source models**: Full transparency including architecture, code, data, and weights (e.g., DeepSeek-R1).

BERT (Bidirectional Encoder Representations from Transformers), released by Google in October 2018 not long after the seminal "Attention is All You Need" paper, pioneered transformers-based models for NLP tasks. Llama-2 and Llama-3 from Meta continued this trend with larger and more efficient architectures. Instruction-tuned models like Alpaca was built Meta's LLaMA model to improve task performance with minimal resources.

| LLM | Number of Parameters | Context Length |
| --- | --- | --- |
| BERT-Base  | 110 million | 512 |
| BERT-Large | 340 million | 512 |
| Llama-2-7B | 7 billion | 4K | 
| Alpaca | 7 billion | 4K |
| Llama-3-8B | 8 billion | 8K | 
| Llama-3-70B | 70 billion | 8K | 
| GPT-2 | 1.5 billion | 1K |
| GPT-3.5 | 175 billion | 4K | 
| GPT-4 | ~1 trillion | 128K|


## DeepSeek-R1 model

**DeepSeek-R1** is a powerful open-weight language model released by DeepSeek in January 2025, with size ranging from from 1.3B to 236B parameters across different variants.  Supporting a context length up to 128K tokens with a GPT-style transformer decoder-only architecture,  it was trained with 6-10T tokens from multilingual internet sources. Furthermore, DeepSeek-R1 was fine-tuned to implement chain-of-thought reasoning without explicit prompting. Its training process included:
- synthetic dataset of thousands of long-form CoT examples
- group relative policy optimization, a reinforcement learning that improved its ability to solve challenging problems
- fine-tuning using a final round of reinforcement learning to boost its reasoning accuracy, helpfulness and harmlessness.

The model exposes its reasoning during inference, a departure from the typical black-box approach of other models, allowing users to witness the model’s "thinking process" as it works through problems.



### Distilled models

Distillation compresses LLMs by transferring knowledge from a large *teacher* model to a smaller *student* model.  
- Knowledge Distillation (KD): Student learns from the teacher’s output probabilities (soft targets) in addition to true labels (hard targets).  
- Intermediate Layer Distillation: Transfers information from internal layers.  
- Data Augmentation: Uses teacher-generated samples to expand the training set.  

LLM distillation is expected to become an even more important practice in the AI world. Examples include GPT-4o distilled into GPT-4o-mini, or DeepSeek-R1 variants trained on Llama and Qwen to preserve reasoning capabilities with fewer parameters.

Distilled versions of DeepSeek-R1 are available in various sizes, including 1.5B, 7B, 14B, 32B, and 70B parameters. These models used DPO (Direct Preference Optimization) or supervised fine-tuning on synthetic highly-curated datasets generated by the larger R1 models, retaining  90–95% of teacher model performance with lower latency.

https://ollama.com/library/deepseek-r1

In [5]:
# model name in Ollama
model_name = "deepseek-r1:14b"

### Small language models

**Small language models (SLMs)** are smaller in scale and scope than large language models (LLMs), with number of parameters ranging from a few million to a few billion. Requiring less memory and computational power, they can be deployed in resource-constrained environments such as edge devices, mobile apps and off-line situations where AI inferencing (when a model generates a response to a user’s query) must be done without a data network.

### Ollama server

Ollama simplifies running open-source LLMs locally. After installing the Ollama runtime and pulling a model (e.g., `deepseek-r1:14b`), it can serve requests on localhost. 
It provides a simple API for creating, running, and managing models, as well as a library of pre-built models. This allows experimentation with high-performance LLMs, improving accessibility, privacy, and latency.

https://github.com/ollama/ollama

1. Install Ollama (https://ollama.com/)
   - `curl https://ollama.ai/install.sh | sh`
   - `ls -ltra `which ollama``
   - `ollama --version`

2. Pull a model (stored in /usr/share/ollama/.ollama/models/)
   - `ollama pull deepseek-r1:14b`
   - `ollama list`

3. Serve an LLM
   - `ollama run deepseek-r1:14b` # uses GPU
   - `ollama ps`

4. or Linux service
   - `sudo systemctl status ollama # service status`
   - `sudo systemctl disable ollama # disable so it does not start up again upon reboot`
   - `sudo systemctl stop ollama # stop service`
   - `sudo systemctl restart ollama # restart service`
   - `sudo rm /etc/systemd/system/ollama.service # delete service file`
   - `sudo rm $(which ollama) # remove ollama binary`

5. Endpoint
   - `curl http://localhost:11434/api/generate -d '{"model": "deepseek-r1:14b", "prompt":"Why is the sky blue?"}'`

In [6]:
# Initializes a local LLM (DeepSeek-R1) using Ollama
from langchain_ollama.llms import OllamaLLM 
model = OllamaLLM(model=model_name, temperature=0)
pprint(model.to_json())

{'id': ['langchain_ollama', 'llms', 'OllamaLLM'],
 'lc': 1,
 'name': 'OllamaLLM',
 'repr': "OllamaLLM(model='deepseek-r1:14b', temperature=0.0)",
 'type': 'not_implemented'}


## Text summarization

Summarization condenses lengthy documents into concise outputs. LLMs can perform abstractive summarization, generating summaries in their own words rather than extracting sentences. Summarization is a core NLP benchmark, critical for a wide variety of applications.

### Natural language processing (NLP) tasks

These tasks play a crucial role in the field of **natural language processing**, challenging research and applications that have enhanced how machines understand and interact with human language.  The performance of LLM's on these tasks are commonly evaluated using large benchmark datasets, such as MMLU (undergraduate level knowledge), GSM-8K (grade-school math), HumanEval (coding), GPQA (graduate-level questions), and MATH (math word problems). However, the intepretation of these results should be tempered by the inadvertent risk that some benchmark examples found their way in the data set used for training models.

- Natural Language Inference (NLI), also known as textual entailment, is the task of determining the relationship between two sentences, i.e. predict whether one sentence (the hypothesis) logically follows from another sentence (the premise).

- Named Entity Recognition (NER) involves identifying and classifying named entities within a text into predefined categories such as person names, organizations, locations, dates, etc.

- Text Generation is the process of generating coherent and contextually relevant text given a certain input or prompt.

- Machine Translation (MT) is the task of automatically translating text from one language to another.

- Text Summarization involves creating a concise summary of a longer text while preserving its key information and meaning.

- Reading comprehension requires models to read a passage of text and answer questions about it, demonstrating understanding of the text. Some challenges when developing and evaluating reading comprehension models include:
  
  - Artifacts, which refer to incorrect or misleading information generated by models that do not reflect the true content of the text but rather exploit patterns in the training data
  - Adversarial attacks, which are instances where models fail due to intentional manipulation or perturbation of the input, aiming to mislead or deceive the model.
  - Multihop reasoning, which refers to the ability of a model to connect multiple pieces of information or "hops" across the text to arrive at an answer.

- Question-Answering (QA) systems that automatically answer questions posed by humans in natural language, either based on a given context or dataset (known as closed-QA) or diverse topics from any domen (open-QA).

- Sentiment Analysis is the task of determining the sentiment or emotional tone expressed in a piece of text, such as positive, negative, or neutral.


### 10-K Market risk disclosures

We focus on Item 7A of the 10-K reports: *Quantitative and Qualitative Disclosures About Market Risk*. After retrieving and filtering disclosures from the SEC’s EDGAR database, only the largest firms with sufficiently long reports are retained. One representative document per sector is selected for summarization.

In [7]:
# Retrieve universe of stocks
beg, end = 20240101, 20240331
univ = crsp.get_universe(bd.endmo(beg, -1))

In [8]:
# lookup company names
comnam = crsp.build_lookup(source='permno', target='comnam', fillna="")
univ['comnam'] = comnam(univ.index)

In [9]:
# lookup sic codes from Compustat, and map to FF 10-sector code
sic = pstat.build_lookup(source='lpermno', target='sic', fillna=0)
industry = Series(sic[univ.index], index=univ.index)
industry = industry.where(industry > 0, univ['siccd'])
sectors = Sectoring(sql, scheme='codes10', fillna='')   # supplement from crosswalk
univ['sector'] = sectors[industry]

In [10]:
# Load Disclosure about Market Risk text from 10-K's
item, form = 'qqr10K', '10-K'
rows = DataFrame(ed.open(form=form, item=item))
found = rows[rows['date'].between(beg, end)]\
    .drop_duplicates(subset=['permno'], keep='last')\
    .set_index('permno')

In [11]:
# Keep largest decile of stocks
found = found.loc[found.index.intersection(univ.index[univ['decile'] == 1])]

In [12]:
# Require minimum length of text
docs = {permno: ed[found.loc[permno, 'pathname']].lower()
        for permno in found.index}
permnos = [permno for permno, doc in docs.items() if len(doc)>2000]
found = found.join(Series(docs, name='item').reindex(permnos), how='inner')
docs = univ.loc[found.index].groupby('sector').sample(1)

### Generation

A LangChain pipeline is used to apply two models (DeepSeek-R1 via Ollama and GPT-4o-mini via OpenAI) to generate summaries. Model endpoints are configured with deterministic settings (temperature = 0). A prompt template and output parser are defined to  extract core content, looping through each 10-K document. Summaries are generated and collected for analysis.

In [13]:
summary = {}  # to collect generated summaries

Define Langchain input prompt template


In [14]:
from langchain_core.prompts import ChatPromptTemplate
prompt_template = """
{role}.
Please summarize this risk report in about 300 words in prose form:

{text}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

Select Langchain output parser


In [15]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

In [16]:
def collect_summaries(model, role="You are a helpful AI assistant."):
    """Helper to iterate over companies and generate summaries of risk reports"""
    summ = {}
    for i, permno in enumerate(docs.index):
        print(f'===== {i+1}/{len(docs)}.', univ.loc[permno, 'comnam'], '=====')
        chain = prompt | model | parser
        response = chain.invoke({"role": role, "text": found.loc[permno, 'item']})
        print("\n".join([textwrap.fill(s, width=80) for s in response.split('\n')]))
        print()
        summ[permno] = response.split('</think>')[-1]   # remove model's "thinking"
    return summ

Generate summaries with DeepSeek-R1-14b model

In [17]:
summary[model_name] = collect_summaries(model)

===== 1/10. PACCAR INC =====
<think>
Okay, so I need to summarize this risk report into about 300 words. Let me read
through it carefully first.

The report is about market risks and derivative instruments, focusing on
interest rates, currencies, and commodities. It mentions that the figures are in
millions. The company uses hedging programs to manage these risks, as described
in Note P.

Starting with interest-rate risk: They measure this by estimating how a 100
basis point increase would affect fair values. In 2023, assets like cash
equivalents and fixed rate loans show potential losses, while liabilities such
as fixed rate term debt and swaps show gains. The total for 2023 is a loss of
$17.7 million, which is better than the previous year's $1.1 million loss.

Next, currency risk: They hedge against several currencies like CAD, EUR, GBP,
etc. A 10% unfavorable change in exchange rates would cause losses of $259.7
million in 2023 and $216.6 million in 2022. But these are offset by ch

Show ollama processes

In [18]:
!ollama ps

NAME               ID              SIZE     PROCESSOR    UNTIL              
deepseek-r1:14b    ea35dfe18182    11 GB    100% GPU     4 minutes from now    


Generate summaries with OpenAI GPT-4o-mini model

In [19]:
summary[gpt_name] = collect_summaries(gpt_model)

===== 1/10. PACCAR INC =====
The risk report outlines the company's exposure to market risks, specifically
focusing on interest rate, currency, and commodity price risks, with figures
presented in millions.

In terms of interest rate risks, the company employs hedging programs to
mitigate exposure to fluctuations. The report quantifies the potential impact of
a 100 basis point increase in interest rates on the fair value of interest-
sensitive assets and liabilities. For 2023, the fair value losses for cash
equivalents and marketable debt securities amounted to $29.2 million, while
fixed-rate loans reflected a loss of $146.5 million. Conversely, fixed-rate term
debt showed gains of $156.8 million, and interest-rate swaps contributed a gain
of $1.2 million, resulting in a total net loss of $17.7 million for the year,
compared to a loss of $1.1 million in 2022.

Regarding currency risks, the company utilizes foreign currency exchange
contracts to hedge against fluctuations in various cur

### Evaluation

__ROUGE__

Recall-Oriented Understudy for Gisting Evaluation (ROUGE) is a set of metrics used to evaluate the quality of summaries by comparing them to reference summaries or human-generated summaries.

- ROUGE-N measures the overlap of n-grams (contiguous sequences of n words) between the system-generated and the reference summaries
- ROUGE-L measures the longest common subsequence (LCS).

__BLEU__
Bilingual Evaluation Understudy (BLEU) evaluates n-gram precision with a brevity penalty to discourage overly short outputs. Originally for machine translation, it is also used for summarization.
- N-gram Precision measures the overlap of n-grams (typically up to 4-grams) between the system-generated summary and the reference summary.
- Brevity Penalty penalizes overly short summaries that do not capture enough information from the reference summaries.
- Cumulative BLEU calculates the geometric mean of BLEU scores for 1-gram to n-gram, rewarding systems that produce more accurate translations across longer phrases.




In [20]:
# These metrics compare overlapping n-grams to measure content similarity.
from rouge_score import rouge_scorer

In [21]:
# computes ROUGE-1 and ROUGE-2 scores between model-generated summaries
def collect_rouge(target, prediction):
    """Helper to loop over companies to compute rouge scores of two risk summaries"""
    scores = {'rouge1': [], 'rouge2': []}
    scorer = rouge_scorer.RougeScorer(scores.keys(), use_stemmer=True)
    for permno in docs.index:
        score = scorer.score(target=target[permno], prediction=prediction[permno])
        for rouge_type in scores.keys():
            scores[rouge_type].append(Series(score[rouge_type]._asdict(),
                                             name=univ.loc[permno, 'comnam']))
    return scores

In [22]:
# Display and compare rouge metric
def display_rouge(rouge_type, scores):
    """Helper to display rouge scores over the companies"""
    df = pd.concat(scores[rouge_type], axis=1)
    print(f"{rouge_type.upper()} metric:")
    return pd.concat([df, df.T.mean().rename('  average')], axis=1).T  # display

In [23]:
# Compute rouge-1 and rouge-2 scores between gpt- and llama-generated summaries
scores = collect_rouge(target=summary[gpt_name], prediction=summary[model_name])

In [24]:
display_rouge("rouge1", scores)

ROUGE1 metric:


,precision,recall,fmeasure
PACCAR INC,0.787234,0.742475,0.764200
PHILLIPS 66,0.366142,0.324042,0.343808
MASTERCARD INC,0.759184,0.628378,0.687616
BRISTOL MYERS SQUIBB CO,0.773913,0.585526,0.666667
CARRIER GLOBAL CORP,0.761628,0.451724,0.567100
LULULEMON ATHLETICA INC,0.560241,0.636986,0.596154
AIRBNB INC,0.720472,0.639860,0.677778
MERCADOLIBRE INC,0.467593,0.331148,0.387716
A T & T INC,0.393805,0.312281,0.348337
REPUBLIC SERVICES INC,0.783410,0.553746,0.648855


In [25]:
display_rouge("rouge2", scores)

ROUGE2 metric:


,precision,recall,fmeasure
PACCAR INC,0.483986,0.456376,0.469775
PHILLIPS 66,0.079051,0.069930,0.074212
MASTERCARD INC,0.381148,0.315254,0.345083
BRISTOL MYERS SQUIBB CO,0.441048,0.333333,0.379699
CARRIER GLOBAL CORP,0.350877,0.207612,0.260870
LULULEMON ATHLETICA INC,0.244713,0.278351,0.260450
AIRBNB INC,0.367589,0.326316,0.345725
MERCADOLIBRE INC,0.116279,0.082237,0.096339
A T & T INC,0.097778,0.077465,0.086444
REPUBLIC SERVICES INC,0.462963,0.326797,0.383142


### Role prompting

By adjusting the system prompt (e.g., “You are a patient teacher”), LLMs can be guided to produce more accessible summaries. This technique, known as **role prompting**, is helpful for tailoring the tone and persona of responses for specific audiences.

In [ ]:
# generates simplified summaries for readability
summary['simple_deepseek'] = collect_summaries(
    model,
    role="You are a patient lower-school teacher, using simple words to explain to your students in the fifth grade.")

===== 1/10. PACCAR INC =====
<think>
Okay, so I need to summarize this risk report for fifth graders. Let me read
through it carefully first.

The report talks about market risks and derivative instruments. It mentions
interest-rate risks, currency risks, and commodity price risks. Each section has
some numbers and explanations.

Starting with interest-rate risks: The company uses hedging programs to manage
how changes in interest rates affect them. They estimate the impact if there's a
100 basis point increase across all yield curves. There are tables showing
potential losses or gains for assets and liabilities in 2023 and 2022.

Next, currency risks: The company hedges against exchange rate fluctuations for
several currencies like Canadian dollar, euro, etc. They mention potential
losses from unfavorable changes in foreign exchange rates, with numbers for 2023
and 2022.

Then, commodity price risks: They use forward contracts to hedge prices of
commodities used in truck production. T

In [32]:
scores = collect_rouge(target=summary[gpt_name], prediction=summary['simple_deepseek'])

In [33]:
display_rouge("rouge1", scores)

ROUGE1 metric:


,precision,recall,fmeasure
PACCAR INC,0.397394,0.408027,0.402640
PHILLIPS 66,0.366142,0.324042,0.343808
MASTERCARD INC,0.371681,0.283784,0.321839
BRISTOL MYERS SQUIBB CO,0.331731,0.226974,0.269531
CARRIER GLOBAL CORP,0.502674,0.324138,0.394130
LULULEMON ATHLETICA INC,0.317172,0.537671,0.398983
AIRBNB INC,0.391304,0.440559,0.414474
MERCADOLIBRE INC,0.467593,0.331148,0.387716
A T & T INC,0.393805,0.312281,0.348337
REPUBLIC SERVICES INC,0.414439,0.504886,0.455213


In [34]:
display_rouge("rouge2", scores)

ROUGE2 metric:


,precision,recall,fmeasure
PACCAR INC,0.133987,0.137584,0.135762
PHILLIPS 66,0.079051,0.069930,0.074212
MASTERCARD INC,0.066667,0.050847,0.057692
BRISTOL MYERS SQUIBB CO,0.053140,0.036304,0.043137
CARRIER GLOBAL CORP,0.134409,0.086505,0.105263
LULULEMON ATHLETICA INC,0.052632,0.089347,0.066242
AIRBNB INC,0.093458,0.105263,0.099010
MERCADOLIBRE INC,0.116279,0.082237,0.096339
A T & T INC,0.097778,0.077465,0.086444
REPUBLIC SERVICES INC,0.150134,0.183007,0.164948


Generate simple summaries with GPT-4o-mini

In [35]:
summary['simple_gpt-4o'] = collect_summaries(
    gpt_model,
    role="You are a patient lower-school teacher, using simple words to explain to your students in the fifth grade.")

===== 1/10. PACCAR INC =====
The risk report talks about how a company manages different types of financial
risks, especially related to market changes. It focuses on three main areas:
interest rates, currency exchange rates, and commodity prices.

First, for interest-rate risks, the company looks at how changes in interest
rates can affect the value of its assets and debts. They estimate what would
happen if interest rates suddenly went up by 1%. In 2023, the company faced
losses of $29.2 million from cash and marketable securities and $146.5 million
from fixed-rate loans. However, they also had gains from fixed-rate debts and
interest-rate swaps, leading to a total loss of $17.7 million, which was worse
than the previous year’s loss of $1.1 million.

Next, the report discusses currency risks. The company uses contracts to protect
itself from changes in foreign currency values, especially with currencies like
the Canadian dollar and the euro. If the value of these currencies drops by 

In [36]:
scores = collect_rouge(target=summary[gpt_name], prediction=summary['simple_gpt-4o'])

Display rouge-1 scores for simple GPT-4o-mini summary

In [37]:
display_rouge("rouge1", scores)

ROUGE1 metric:


,precision,recall,fmeasure
PACCAR INC,0.653571,0.612040,0.632124
PHILLIPS 66,0.551971,0.536585,0.544170
MASTERCARD INC,0.560261,0.581081,0.570481
BRISTOL MYERS SQUIBB CO,0.550000,0.542763,0.546358
CARRIER GLOBAL CORP,0.551155,0.575862,0.563238
LULULEMON ATHLETICA INC,0.576577,0.657534,0.614400
AIRBNB INC,0.658621,0.667832,0.663194
MERCADOLIBRE INC,0.553333,0.544262,0.548760
A T & T INC,0.665480,0.656140,0.660777
REPUBLIC SERVICES INC,0.650794,0.667752,0.659164


Display rouge-2 scores for simple GPT-4o-mini summary

In [38]:
display_rouge("rouge2", scores)

ROUGE2 metric:


,precision,recall,fmeasure
PACCAR INC,0.336918,0.315436,0.325823
PHILLIPS 66,0.244604,0.237762,0.241135
MASTERCARD INC,0.241830,0.250847,0.246256
BRISTOL MYERS SQUIBB CO,0.257525,0.254125,0.255814
CARRIER GLOBAL CORP,0.231788,0.242215,0.236887
LULULEMON ATHLETICA INC,0.243976,0.278351,0.260032
AIRBNB INC,0.359862,0.364912,0.362369
MERCADOLIBRE INC,0.254181,0.250000,0.252073
A T & T INC,0.257143,0.253521,0.255319
REPUBLIC SERVICES INC,0.382166,0.392157,0.387097


### Readability

Readability scores such as **Flesch-Kincaid** and **Gunning-Fog** assess how easy a summary is to read. These metrics are calculated based on sentence length, word complexity, and syllable count, and correspond to U.S. grade levels. Simpler summaries which score lower are suitable for broader audiences.

In [39]:
# applies Flesch-Kincaid and Gunning-Fog readability indexes to measure how complex each summary is
from readability import Readability
fog = {permno: {name: Readability(summary[name][permno]).flesch_kincaid().grade_level
                for name in ['simple_deepseek', 'simple_gpt-4o', model_name, gpt_name]}
       for permno in summary[gpt_name].keys()}
DataFrame(fog).T  # display grade-level

,simple_deepseek,simple_gpt-4o,deepseek-r1:14b,gpt-4o-mini
60506,7,9,13,15
13356,12,10,12,17
91233,10,10,16,16
19393,9,10,15,15
19285,8,10,17,16
92203,9,11,15,15
20190,9,12,15,17
92221,12,10,12,16
66093,16,12,16,16
86228,9,8,10,12


In [40]:
fog = {permno: {name: Readability(summary[name][permno]).gunning_fog().grade_level
                for name in ['simple_deepseek', 'simple_gpt-4o', model_name, gpt_name]}
       for permno in summary[gpt_name].keys()}
DataFrame(fog).T  # display grade-level

,simple_deepseek,simple_gpt-4o,deepseek-r1:14b,gpt-4o-mini
60506,9,12,college_graduate,college_graduate
13356,college,college,college,college_graduate
91233,12,12,college_graduate,college_graduate
19393,12,college,college_graduate,college_graduate
19285,11,college,college_graduate,college_graduate
92203,12,college,college_graduate,college_graduate
20190,12,college,college_graduate,college_graduate
92221,12,college,12,college_graduate
66093,college_graduate,college,college_graduate,college_graduate
86228,12,12,college,college


**References:**

Greg Durrett, 2021-2024, "CS388 Natural Language Processing course materials", retrieved from https://www.cs.utexas.edu/~gdurrett/courses/online-course/materials.html

Philipp Krähenbühl, 2020-2024, "AI394T Deep Learning course materials", retrieved from
https://www.philkr.net/dl_class/material and https://ut.philkr.net/deeplearning/

Philipp Krähenbühl, 2025, "AI395T Advances in Deep Learning course materials", retrieved from https://ut.philkr.net/advances_in_deeplearning/